In [16]:
import os
import numpy as np
import talib as ta
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm

In [17]:
path = "Z:\data\BTCUSDT\BTCUSDT-1h"

In [18]:
filelist = os.listdir(path)
filelist.sort()
df = pd.DataFrame()
for file in tqdm(filelist[-400:]):
    df1 = pd.read_csv(os.path.join(path, file))
    df = pd.concat([df, df1])
df

100%|██████████| 400/400 [00:02<00:00, 150.36it/s]


,open_time,open,high,low,close,volume,close_time,quote_volume,count,taker_buy_volume,taker_buy_quote_volume,ignore
0,1662336000000,19992.7,20046.7,19865.0,19906.4,24500.011,1662339599999,4.886476e+08,140742,12409.292,2.475152e+08,0
1,1662339600000,19906.4,19958.8,19801.0,19831.8,17846.297,1662343199999,3.548402e+08,104290,7599.590,1.511258e+08,0
2,1662343200000,19831.8,19856.1,19803.4,19842.1,11111.015,1662346799999,2.203534e+08,71809,6270.538,1.243638e+08,0
3,1662346800000,19842.1,19894.1,19803.3,19871.9,10370.516,1662350399999,2.059306e+08,67489,5008.919,9.947083e+07,0
4,1662350400000,19871.9,19900.0,19827.3,19878.0,8357.326,1662353999999,1.660060e+08,58922,3705.192,7.360250e+07,0
...,...,...,...,...,...,...,...,...,...,...,...,...
19,1696878000000,27555.0,27632.4,27526.3,27607.6,8683.886,1696881599999,2.394986e+08,94387,4654.296,1.283765e+08,0
20,1696881600000,27607.7,27644.0,27542.9,27556.2,4995.344,1696885199999,1.378315e+08,62913,2328.249,6.424873e+07,0
21,1696885200000,27556.1,27627.2,27542.3,27607.4,3540.588,1696888799999,9.768003e+07,48596,1892.288,5.220956e+07,0
22,1696888800000,27607.4,27627.0,27562.0,27575.9,3771.446,1696892399999,1.040506e+08,49937,1651.572,4.556535e+07,0


In [54]:
df1 = df[['open_time', 'open', 'high', 'low', 'close', 'volume']]
df1.columns = ['t', 'o', 'h', 'l', 'c', 'v']
df1 = df1.dropna()
df1

,t,o,h,l,c,v
0,1662336000000,19992.7,20046.7,19865.0,19906.4,24500.011
1,1662339600000,19906.4,19958.8,19801.0,19831.8,17846.297
2,1662343200000,19831.8,19856.1,19803.4,19842.1,11111.015
3,1662346800000,19842.1,19894.1,19803.3,19871.9,10370.516
4,1662350400000,19871.9,19900.0,19827.3,19878.0,8357.326
...,...,...,...,...,...,...
19,1696878000000,27555.0,27632.4,27526.3,27607.6,8683.886
20,1696881600000,27607.7,27644.0,27542.9,27556.2,4995.344
21,1696885200000,27556.1,27627.2,27542.3,27607.4,3540.588
22,1696888800000,27607.4,27627.0,27562.0,27575.9,3771.446


In [55]:
# 简单算术平均
def Simple_arithmetic_average(df, length):
    df = df.copy()
    df['saap'] = df['c'].rolling(length).mean()
    return df['saap']


# 成交量加权平均(VWAP)
def vwap(df, length):
    df = df.copy()
    df['sumv'] = df['v'].rolling(length).mean()
    df['Mulpv'] = df['c'] * df['v']
    df['sumpv'] = df['Mulpv'].rolling(length).mean()
    df['vwap'] = df['sumpv'] / df['sumv']
    return df['vwap']


# OHLC四价平均
def OHLC(df, length):
    df = df.copy()
    df['OHLC'] = (df['o'] + df['h'] + df['l'] + df['c']) / 4
    return df['OHLC']

In [56]:
length = 9
df1['saap'] = Simple_arithmetic_average(df=df1, length=9)
df1['vwap'] = vwap(df=df1, length=9)
df1['OHLC'] = OHLC(df=df1, length=9)

In [57]:
df1 = df1[length - 1:]
df1

,t,o,h,l,c,v,saap,vwap,OHLC
8,1662364800000,19739.9,19780.6,19712.8,19767.6,10909.679,19813.533333,19800.253720,19750.225
9,1662368400000,19767.6,19794.7,19720.0,19754.8,12981.123,19796.688889,19777.214773,19759.275
10,1662372000000,19754.8,19777.7,19651.0,19686.3,22336.892,19780.522222,19756.198407,19717.450
11,1662375600000,19686.4,19715.2,19620.2,19692.9,22456.001,19763.944444,19740.802978,19678.675
12,1662379200000,19692.9,19749.0,19659.5,19715.9,17205.839,19746.611111,19729.708234,19704.325
...,...,...,...,...,...,...,...,...,...
19,1696878000000,27555.0,27632.4,27526.3,27607.6,8683.886,27493.633333,27476.685860,27580.325
20,1696881600000,27607.7,27644.0,27542.9,27556.2,4995.344,27504.777778,27481.274719,27587.700
21,1696885200000,27556.1,27627.2,27542.3,27607.4,3540.588,27525.011111,27491.721048,27583.250
22,1696888800000,27607.4,27627.0,27562.0,27575.9,3771.446,27532.811111,27492.440138,27593.075


In [58]:
df1.to_csv('Average Price.csv', index=False)